# Describing the population of grain sizes

The method to describe the properties of the grain size population is named ``summarize()``. Before we get into the details of the method, let's run the GrainSizeTools script, load the example dataset, and create a toy dataset with known parameters.

In [1]:
# Load the script first (change the path to GrainSizeTools_script.py accordingly!)
%run C:/Users/marco/Documents/GitHub/GrainSizeTools/grain_size_tools/GrainSizeTools_script.py

module plot imported
module averages imported
module stereology imported
module piezometers imported
module template imported

Welcome to GrainSizeTools script
A free open-source cross-platform script to visualize and characterize grain size
population and estimate differential stress via paleopizometers.

Version: v3.0RC0 (2020-04-23)
Documentation: https://marcoalopez.github.io/GrainSizeTools/

Type get.functions_list() to get a list of the main methods



In [2]:
# Load the example dataset
filepath = 'C:/Users/marco/Documents/GitHub/GrainSizeTools/grain_size_tools/DATA/data_set.txt'
dataset = pd.read_csv(filepath, sep='\t')

# estimate equivalent circular diameters (ECDs)
dataset['diameters'] = 2 * np.sqrt(dataset['Area'] / np.pi)
dataset.head()

,,Area,Circ.,Feret,FeretX,FeretY,FeretAngle,MinFeret,AR,Round,Solidity,diameters
0,1,157.25,0.680,18.062,1535.0,0.5,131.634,13.500,1.101,0.908,0.937,14.149803
1,2,2059.75,0.771,62.097,753.5,16.5,165.069,46.697,1.314,0.761,0.972,51.210889
2,3,1961.50,0.842,57.871,727.0,65.0,71.878,46.923,1.139,0.878,0.972,49.974587
3,4,5428.50,0.709,114.657,1494.5,83.5,19.620,63.449,1.896,0.528,0.947,83.137121
4,5,374.00,0.699,29.262,2328.0,34.0,33.147,16.000,1.515,0.660,0.970,21.821815


In [3]:
# Set the population properties
scale = np.log(20)  # set sample geometric mean to 20
shape = np.log(1.5)  # set the lognormal shape to 1.5

# generate a random lognormal population of size 500
np.random.seed(seed=1)  # this is to generate always the same population for reproducibility
toy_dataset = np.random.lognormal(mean=scale, sigma=shape, size=500)
print('sample size =', len(toy_dataset))

sample size = 500


We are now ready to check what we can get from the function ``summarize()``. The simplest example of use would be to pass the data containing the diameters. For simplicity's sake, let's do it with the toy dataset first.

In [4]:
summarize(toy_dataset)

 
CENTRAL TENDENCY ESTIMATORS
Arithmetic mean = 22.13 microns
Confidence intervals at 95.0 %
mCox method: 21.35 - 22.98 (-3.5%, +3.8%), length = 1.623
Geometric mean = 20.44 microns
Confidence interval at 95.0 %
CLT method: 19.73 - 21.17 (-3.5%, +3.6%), length = 1.441
Median = 20.32 microns
Confidence interval at 95.0 %
robust method: 19.33 - 21.42 (-4.9%, +5.4%), length = 2.096
Mode (KDE-based) = 17.66 microns
Maximum precision set to 0.1
KDE bandwidth = 2.78 (silverman rule)
 
DISTRIBUTION FEATURES
Sample size (n) = 500
Standard deviation = 9.07 (1-sigma)
Interquartile range (IQR) = 11.44
Lognormal shape (Multiplicative Standard Deviation) = 1.49
Shapiro-Wilk test warnings:
Data is not normally distributed!
Normality test: 0.88, 0.00 (test statistic, p-value)


By default, the ```summarize()``` function returns:

- Different **central tendency estimators** ("averages") including the arithmetic and geometric means, the median, and the KDE-based mode (i.e. frequency peak).
- The **confidence intervals** for the different means and the median at 95% of certainty in absolute value and percentage relative to the average (*a.k.a* coefficient of variation). The meaning of these intervals is that, given the observed data, there is a 95% probability (one in 20) that the true value of grain size falls within this credible interval. The function provides the lower and upper bounds of the confidence interval, the error in percentage respect to the average, and the interval length. 
- The methods used to estimate the confidence intervals for each average (excepting for the mode). The function ```summarize()``` automatically choose the optimal method depending on distribution features (see below)
- The sample size and two population dispersion measures: the (Bessel corrected) [standard deviation](https://en.wikipedia.org/wiki/Standard_deviation) and the [interquartile range](https://en.wikipedia.org/wiki/Interquartile_range).
- The shape of the lognormal distribution using the multiplicative standard deviation (MSD)
- A Shapiro-Wilk test warning indicating when the data deviates from normal and/or lognormal (when p-value < 0.05).

In the example above, the Shapiro-Wilk test tells us that the distribution is not normally distributed, which is to be expected since we know that this is a lognormal distribution. Note that the geometric mean and the lognormal shape are very close to the values used to generate the synthetic random dataset, 20 and 1.5 respectively. Now, let's do the same using the dataset that comes from a real rock, for this, we have to pass the column with the diameters:

In [5]:
summarize(dataset['diameters'])

 
CENTRAL TENDENCY ESTIMATORS
Arithmetic mean = 34.79 microns
Confidence intervals at 95.0 %
CLT (ASTM) method: 34.09 - 35.48, (±2.0%), length = 1.393
Geometric mean = 30.10 microns
Confidence interval at 95.0 %
CLT method: 29.47 - 30.75 (-2.1%, +2.2%), length = 1.283
Median = 31.53 microns
Confidence interval at 95.0 %
robust method: 30.84 - 32.81 (-2.2%, +4.1%), length = 1.970
Mode (KDE-based) = 24.31 microns
Maximum precision set to 0.1
KDE bandwidth = 4.01 (silverman rule)
 
DISTRIBUTION FEATURES
Sample size (n) = 2661
Standard deviation = 18.32 (1-sigma)
Interquartile range (IQR) = 23.98
Lognormal shape (Multiplicative Standard Deviation) = 1.75
Shapiro-Wilk test warnings:
Data is not normally distributed!
Normality test: 0.94, 0.00 (test statistic, p-value)
Data is not lognormally distributed!
Lognormality test: 0.99, 0.03 (test statistic, p-value)


Leaving aside the different numbers, there are some subtle differences compared to the results obtained with the toy dataset. First, the confidence interval method for the arithmetic mean is no longer the modified Cox (mCox) but the one based on the central limit theorem (CLT) advised by the [ASTM](https://en.wikipedia.org/wiki/ASTM_International). As previously noted, the function ```summarize()``` automatically choose the optimal confidence interval method depending on distribution features. We show below the decision tree flowchart for choosing the optimal confidence interval estimation method, which is based on [Lopez-Sanchez (2020)](https://doi.org/10.1016/j.jsg.2020.104042).

![](https://github.com/marcoalopez/GrainSizeTools/blob/master/FIGURES/avg_map.png?raw=true)

The reason why the CLT method applies in this case is that the grain size distribution is not sufficiently close to a logarithmic distribution (note the Shapiro-Wilk test warning with a p-value < 0.05), and this might cause an inaccurate estimate of the arithmetic mean confidence interval.

Now, let's focus on the different options of the ``summarize()`` method.

In [6]:
?summarize

Signature:
summarize(
    data,
    avg=('amean', 'gmean', 'median', 'mode'),
    ci_level=0.95,
    bandwidth='silverman',
    precision=0.1,
)
Docstring:
Estimate different grain size statistics. This includes different means,
the median, the frequency peak grain size via KDE, the confidence intervals
using different methods, and the distribution features.

Parameters
----------
data : array_like
    the size of the grains

avg : string, tuple or list; optional
    the averages to be estimated

    | Types:
    | 'amean' - arithmetic mean
    | 'gmean' - geometric mean
    | 'median' - median
    | 'mode' - the kernel-based frequency peak of the distribution

ci_level : scalar between 0 and 1; optional
    the certainty of the confidence interval (default = 0.95)

bandwidth : string {'silverman' or 'scott'} or positive scalar; optional
    the method to estimate the bandwidth or a scalar directly defining the
    bandwidth. It uses the Silverman plug-in method by default.

precision 

> **TODO:**
- explain the different options of ``summarize()`` through examples
- examples using log-transformed populations